In [1]:
%reset
import pandas as pd
import re
import string
import unicodedata
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from pathlib import Path
from textblob import TextBlob
nltk.download('stopwords')
pd.options.mode.chained_assignment = None 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class DataProcessor:
    def __init__(self, data):
        self.data = data
        self.n_tweet = 0  # Nombre de tweets  
        self.n_tweet_doublons = 0 # Nombre de tweets doublons 
        self.n_de_mots = 0 # Nombre de mots dans tous les tweets
        self.n_urls = 0  # Nombre de liens supprimés      
        self.n_mentions = 0 # Nombre de mentions supprimés 
        self.n_hashtags = 0 # Nombre de hashtags supprimés 
        self.n_stopwords = 0 # Nombre de stopwords supprimés
        self.entreprises = ["LVMH", "L'Oréal", "Hermès", "TotalEnergies", 
                            "Sanofi", "Airbus", "Schneider Electric", "Capgemini", 
                            "Air liquide", "BNP Paribas"]
 
    def conservation_top_10(self):
        """
            Conservation des 10 plus grosses entreprises du CAC40
        """
        self.data = self.data[self.data.entreprise.isin(self.entreprises)]
        self.n_tweet = len(self.data)

    def compter_mots(self):
        def compter_mots_chaque_ligne(text):
            words = re.split(r"[ ’'\-]", text)
            return len(words)
        
        self.n_de_mots = self.data['text'].apply(compter_mots_chaque_ligne).sum()

    def supprime_urls_hashtags_mentions_ponctuations_nombres(self):
        """
        Supprime les liens https jusqu'à l'espace suivant
        Supprime toutes les mentions (@) ainsi que la suite jusqu'à l'espace suivant
        Supprime tous les Hashtags (#) ainsi que la suite jusqu'à l'espace suivant 
        Supprime tous les nombres
        Supprime tous les pontuations
        Supprime tous les accents
        """
        url_regex = r'(http[s]?:\/\/\S+)'
        pattern = r'(@\w+|#\w+|http[s]?:\/\/\S+)'
        num_regex = r'\d+'
        
        def clean(text):
            items = re.findall(pattern, text)
            mentions = [item for item in items if item.startswith('@')]
            hashtags = [item for item in items if item.startswith('#')]
            urls = [item for item in items if re.match(url_regex, item)]         
            n_mentions = len(mentions)
            n_hashtags = len(hashtags)
            n_urls = len(urls)    
            text = re.sub(pattern, '', text)
            text = re.sub(num_regex, '', text)
            punctuations = re.findall(r'[^\w\s]', text)  
            n_pontuations = len(punctuations)
            # Ice, faut remplace par l'espace 
            text = re.sub(r'[^\w\s]', ' ', text)
                
            return text, mentions, hashtags, urls,punctuations, n_mentions, n_hashtags, n_urls, n_pontuations

        self.data['text'], self.data['mentions'], self.data['hashtags'], self.data['urls'], self.data['punctuations'], self.n_mentions, self.n_hashtags, self.n_urls,self.n_pontuations = zip(*self.data['text'].apply(clean))
        self.data['text'] = self.data['text'].str.lower()
        self.n_mentions = sum(self.n_mentions)
        self.n_hashtags = sum(self.n_hashtags)
        self.n_urls = sum(self.n_urls)
        self.n_punctuations = sum(self.n_pontuations)
        def remove_accents(text):
            return unidecode(text)
        self.data['text'] = self.data['text'].apply(remove_accents)

    def supprime_les_stopwords(self):
        nltk.download('stopwords')
        stop_words = set(stopwords.words('french'))
        liste_mot_non_voulu = ["lvmh", "oreal","oréal","L’Oréal", "sanofi", "airbus", "totalenergie", "totalenergies", "total", "energie", "energies", "air", "liquide", "bnp", "paribas", "pariba", "airliquide", "ça", "ca", "faire", "moi", "deja"]
        stop_words_txt = pd.read_table("data/stop_words_french.txt", header = 0)
        list_sw = stop_words_txt["vides"].to_list()
        stop_words.update(liste_mot_non_voulu)
        stop_words.update(list_sw)
        
        def supprime_stopwords(text):
            words = re.split(r"[ ’'\-]", text)
            filtered_words = [word for word in words if word.lower() not in stop_words]
            filtered_stop_words = [word for word in words if word.lower() in stop_words]
            return ' '.join(filtered_words),filtered_stop_words, len(words) - len(filtered_words)

        self.data['text'],self.data["stopword"], n_stopwords = zip(*self.data['text'].apply(supprime_stopwords))
        self.n_stopwords = sum(n_stopwords)
    
    def transform_normal(self):
        """       
            Supprime tous les accents     
            Supprime les tweets n'ayant plus de texte. 
        """
        self.data['text'] = self.data['text'].str.strip()
        self.data = self.data[self.data['text'] != '']
        self.data['date'] = pd.to_datetime(self.data['date']).dt.date
        
    def supprime_doublons(self):
        """
            Supprime tous les doublons avec le même texte
        """
        self.data = self.data.drop_duplicates(["text"])
        self.n_tweet_sans_doublons = len(self.data)
        self.n_tweet_doublons = self.n_tweet - self.n_tweet_sans_doublons
    
    def stats_desc(self):
        """
            Retourne un dataframe contenant les résultats
        """
        results_dict = {
            "Nombre de tweets": [self.n_tweet],           
            "Nombre de liens": [self.n_urls],
            "Nombre de mots":[self.n_de_mots],
            "Nombre de stop word":[self.n_stopwords],
            "Nombre de mentions": [self.n_mentions],
            "Nombre de hashtags": [self.n_hashtags],
            "Nombre de tweets doublons": [self.n_tweet_doublons]

            }
        self.results_dict = pd.DataFrame(results_dict)
        return self.results_dict
    
    def lemmatize(self, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
        nlp = spacy.load("fr_core_news_sm", disable=["parser", "ner"])
        self.data['text'] = self.data['text'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if token.pos_ in allowed_postags]))
        
    def sentiment_analysis(self):
        def getSubjectivity(text):
            """ 
                La propriété sentiment d'un objet TextBlob renvoie un tuple nommé de la 
                forme (subjectivité), où la subjectivité est un flottant compris entre 0,0 
                et 1,0, indiquant la subjectivité du texte. Une subjectivité de 0,0 signifie 
                que le texte est très objectif et factuel, tandis qu'une subjectivité de 
                1,0 signifie que le texte est très subjectif et opiniâtre.
            """
            return TextBlob(text).sentiment.subjectivity


        def getPolarity(text):
            """
                La propriété sentiment d'un objet TextBlob renvoie un tuple nommé de la forme 
                (polarity ), où polarity est un flottant compris entre -1.0 et 1.0, indiquant
                la polarité de sentiment du texte. Une polarité de -1,0 est très négative,
                0 est neutre et 1,0 est très positive.
            """
            return TextBlob(text).sentiment.polarity

        self.data["Subjectivité"] = self.data["text"].apply(getSubjectivity)
        self.data["Polarité"] = self.data["text"].apply(getPolarity)


        def getAnalysis_polarity(score):
            if score < 0:
                return "Negative"
            elif score == 0:
                return "Neutral"
            else:
                return "Positive"
        def getAnalysis_subjectivity(score):
            if score < 0.5:
                return "Très objectif et factuel"
            else:
                return "Très subjectif et opiniâtre"
        self.data["Subjectivité"] = self.data["Subjectivité"].apply(getAnalysis_subjectivity)
        self.data["Polarité"] = self.data["Polarité"].apply(getAnalysis_polarity)

In [3]:
def transform(data):  
    data_processor = DataProcessor(data)
    data_init = data_processor.data.copy()
    data_processor.conservation_top_10()
    data_processor.compter_mots()
    data_processor.supprime_urls_hashtags_mentions_ponctuations_nombres()
    #data_processor.lemmatize() # Dans le cas que on veut lemmatizer les données 
    data_processor.supprime_les_stopwords()
    data_processor.transform_normal()
    data_processor.supprime_doublons()
    data_processor.sentiment_analysis()
    data_fin = data_processor.data
    data_processor.stats_desc()
    results = data_processor.results_dict
    return data_init, data_fin, results 

In [4]:
path = Path.cwd().joinpath('data')
path_data = path.joinpath('df.tweet.gzip')
data = pd.read_parquet(path_data) 
data_init, data_fin, results = transform(data)
print(len(data_init))
print(len(data_fin))
print(results)
# data_init.to_parquet('data/data_init.parquet')
# data_fin.to_parquet('data/data_fin.parquet')
# results.to_parquet('data/results.parquet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


43518
19177
   Nombre de tweets  Nombre de liens  Nombre de mots  Nombre de stop word  \
0             23531            14670          604278               274714   

   Nombre de mentions  Nombre de hashtags  Nombre de tweets doublons  
0               39081               14883                       4354  


In [213]:
import gensim
from collections import Counter

class Stats_desc:
    def __init__(self, data):
        self.data = data
        self.n_tweet = 0  # Nombre de tweets  
        self.n_de_mots = 0 # Nombre de mots dans tous les tweets
        self.n_urls = 0  # Nombre de liens supprimés      
        self.n_mentions = 0 # Nombre de mentions supprimés 
        self.n_hashtags = 0 # Nombre de hashtags supprimés 
        self.n_stopwords = 0 # Nombre de stopwords supprimés
        self.data_stats = None # Les dataframes statistiques dé données final
        self.data_text = None # Les lists des lists tous les mots 
        
    def stats(self):  
        def counts(data):     
            word_counts = [len(s.split()) for s in self.data['text'].tolist()]
            count = pd.DataFrame({'Entreprise': self.data["entreprise"],
                                  'Nombre de tweets': 1,
                                  'Nombre de mots': word_counts,
                                  'Mentions': self.data["mentions"].apply(len),
                                  'Hashtags': self.data["hashtags"].apply(len),
                                  'Urls': self.data["urls"].apply(len),
                                  'Ponctuations': self.data["punctuations"].apply(len),
                                  'Stopwords': self.data["stopword"].apply(len)})
            return count
        self.data_stats = counts(self.data)
        return self.data_stats 
    
    def frequence(self, top_n):
        def tokenize(docs):
            tokenized_docs = []
            for doc in docs:
                tokens = gensim.utils.simple_preprocess(doc, deacc=True)
                tokens = [token for token in tokens if len(token) > 1]
                tokenized_docs.append(tokens)
            return tokenized_docs
    
        data_grouped = self.data.groupby('entreprise')['text'].apply(tokenize)
        
        # Count the frequency of each word for each entreprise
        word_counts = data_grouped.apply(lambda x: Counter(word for doc in x for word in doc))
        
        # Concatenate the word counts into a single DataFrame
        dfs = []
        for entreprise in word_counts.index:
            counts = pd.DataFrame.from_dict(word_counts[entreprise], orient='index', columns=['count'])
            counts['Entreprise'] = entreprise
            dfs.append(counts)
        df = pd.concat(dfs)
        
        # Sort the DataFrame by count in descending order and get the top n rows for each entreprise
        df = df.sort_values(['Entreprise', 'count'], ascending=[True, False]).groupby('Entreprise').head(top_n)
        
        # Convert the index to a column and reset the index
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'mots'}, inplace=True)
        self.frequence_mots = df
        return self.frequence_mots

In [218]:
data_stats = Stats_desc(data_fin)
data_stats.stats()
data_stats.frequence(10)

,mots,count,Entreprise
0,hydrogene,50,Air liquide
1,france,27,Air liquide
2,actions,22,Air liquide
3,stations,22,Air liquide
4,gaz,21,Air liquide
...,...,...,...
95,edf,168,TotalEnergies
96,francais,150,TotalEnergies
97,contrat,149,TotalEnergies
98,projet,140,TotalEnergies
